In [30]:
from keras.models import Model, load_model
from keras.layers import Input, Convolution1D, Dot, Dense, Activation, Concatenate
from keras.utils import Sequence
import numpy as np
import random
import json
from typing import List, Dict, Tuple

## Load the model

In [40]:
with open('options.json', encoding='utf-8') as f:
    options = json.load(f)
print(options)

{'input_token_index': {' ': 0, '!': 1, '#': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '<': 27, '=': 28, '>': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '\\': 58, '^': 59, '_': 60, '`': 61, 'a': 62, 'b': 63, 'c': 64, 'd': 65, 'e': 66, 'f': 67, 'g': 68, 'h': 69, 'i': 70, 'j': 71, 'k': 72, 'l': 73, 'm': 74, 'n': 75, 'o': 76, 'p': 77, 'q': 78, 'r': 79, 's': 80, 't': 81, 'u': 82, 'v': 83, 'w': 84, 'x': 85, 'y': 86, 'z': 87, '~': 88, '¡': 89, '¢': 90, '£': 91, '¤': 92, '¥': 93, '¦': 94, '§': 95, '¨': 96, '©': 97, '¬': 98, '®': 99, '¯': 100, '°': 101, '±': 102, '³': 103, '´': 104, 'µ': 105, '¸': 106, '¹': 107, 'º': 1

In [6]:
latent_dim = 256 

In [17]:
def create_model():
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, options['num_encoder_tokens']))
    # Encoder
    x_encoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal')(encoder_inputs)
    x_encoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal', dilation_rate=2)(x_encoder)
    x_encoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal', dilation_rate=4)(x_encoder)
    
    decoder_inputs = Input(shape=(None, options['num_decoder_tokens']))
    # Decoder
    x_decoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal')(decoder_inputs)
    x_decoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal', dilation_rate=2)(x_decoder)
    x_decoder = Convolution1D(256, kernel_size=3, activation='relu',
                              padding='causal', dilation_rate=4)(x_decoder)
    # Attention
    attention = Dot(axes=[2, 2])([x_decoder, x_encoder])
    attention = Activation('softmax')(attention)
    
    context = Dot(axes=[2, 1])([attention, x_encoder])
    decoder_combined_context = Concatenate(axis=-1)([context, x_decoder])
    
    decoder_outputs = Convolution1D(64, kernel_size=3, activation='relu',
                                    padding='causal')(decoder_combined_context)
    decoder_outputs = Convolution1D(64, kernel_size=3, activation='relu',
                                    padding='causal')(decoder_outputs)
    # Output
    decoder_dense = Dense(options['num_decoder_tokens'], activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.summary()
    return model

In [31]:
#model = create_model()
#model.load_weights('s2s.h5')
model = load_model('s2s.h5')
#model.compile(optimizer='adam', loss='categorical_crossentropy')

In [61]:
user_inputs = [
    "There are numerous weaknesses with the bag of words model especially when applied to natural language processing tasks that graph ranking algorithms such as TextRank are able to address.",
    "Since purple yams happen to be starchy root vegetables, they also happen to be a great source of carbs, potassium, and vitamin C.",
    "Recurrent Neural Networks (RNNs) have been used successfully for many tasks involving sequential data such as machine translation, sentiment analysis, image captioning, time-series prediction etc.",
    "Improved RNN models such as Long Short-Term Memory networks (LSTMs) enable training on long sequences overcoming problems like vanishing gradients.",
    "However, even the more advanced models have their limitations and researchers had a hard time developing high-quality models when working with long data sequences.",
    "In machine translation, for example, the RNN has to find connections between long input and output sentences composed of dozens of words.",
    "It seemed that the existing RNN architectures needed to be changed and adapted to better deal with such tasks.",
    "Wenger ended his 22-year Gunners reign after the 2017-18 season and previously stated he intended to take charge of a new club in early 2019.",
    "It will not prevent the Frenchman from resuming his career in football.",
    "However 12 months out of work has given him a different outlook and may influence his next move.",
]

In [68]:
def user_input_to_inputs(ui: List[str]):
    max_encoder_seq_length = options['max_encoder_seq_length']
    num_encoder_tokens = options['num_encoder_tokens']
    input_token_index = options['input_token_index']
    encoder_input_data = np.zeros(
        (len(ui), max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')

    for i, input_text in enumerate(ui):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
       
    return encoder_input_data

In [69]:
inputs = user_input_to_inputs(user_inputs)

In [70]:
def print_predictions(inputs:np.array, user_inputs: List[str]):
    max_decoder_seq_length = options['max_decoder_seq_length']
    num_decoder_tokens = options['num_decoder_tokens']
    input_token_index = options['input_token_index']
    target_token_index = options['target_token_index']
    
    # Define sampling models
    reverse_input_char_index = dict(
        (i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict(
        (i, char) for char, i in target_token_index.items())

    in_encoder = inputs
    in_decoder = np.zeros(
        (len(in_encoder), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')

    in_decoder[:, 0, target_token_index["\t"]] = 1

    predict = np.zeros(
        (len(in_encoder), max_decoder_seq_length),
        dtype='float32')

    for i in range(max_decoder_seq_length - 1):
        predict = model.predict([in_encoder, in_decoder])
        predict = predict.argmax(axis=-1)
        predict_ = predict[:, i].ravel().tolist()
        for j, x in enumerate(predict_):
            in_decoder[j, i + 1, x] = 1

    for seq_index in range(len(in_encoder)):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        output_seq = predict[seq_index, :].ravel().tolist()
        decoded = []
        for x in output_seq:
            if reverse_target_char_index[x] == "\n":
                break
            else:
                decoded.append(reverse_target_char_index[x])
        decoded_sentence = "".join(decoded)
        print('-')
        print('Input sentence:', user_inputs[seq_index])
        print('Decoded sentence:', decoded_sentence)

In [71]:
print_predictions(inputs, user_inputs)

-
Input sentence: There are numerous weaknesses with the bag of words model especially when applied to natural language processing tasks that graph ranking algorithms such as TextRank are able to address.
Decoded sentence: Graph ranking are able to address.
-
Input sentence: Since purple yams happen to be starchy root vegetables, they also happen to be a great source of carbs, potassium, and vitamin C.
Decoded sentence: Since purple happen to be a great source of carbs.
-
Input sentence: Recurrent Neural Networks (RNNs) have been used successfully for many tasks involving sequential data such as machine translation, sentiment analysis, image captioning, time-series prediction etc.
Decoded sentence: Recurrent Neural Networks.
-
Input sentence: Improved RNN models such as Long Short-Term Memory networks (LSTMs) enable training on long sequences overcoming problems like vanishing gradients.
Decoded sentence: Improved RNN models sequences.
-
Input sentence: However, even the more advanced 